In [1]:
import csv
import pandas as pd
import numpy as np
import networkx as nx
import sympy
import math
import matplotlib.pyplot as plt 
import knowledge_representation as KR
import itertools
import re

generating the knowledge graph that we previously built, to implement our applications on

In [2]:
G = KR.generate_graph(KR.import_equations(), KR.import_variables())
print([x for x in G.nodes()])
print([(x,y,z) for x,y,z in G.edges(data = True)])

['A', 'a', 'A_vec', 'alpha', 'B', 'beta', 'Bx', 'By', 'Bz', 'c', 'C', 'chi', 'd', 'D', 'd1', 'd2', 'delta', 'el_ct', 'E_n', 'E_den', 'Ef', 'epsilon', 'f', 'F', 'flux', 'foc', 'G', 'g', 'g_', 'gamma', 'H', 'h', 'hbar', 'H_G', 'I', 'I_0', 'I1', 'I2', 'Int', 'Int_0', 'j', 'Jz', 'K', 'k', 'k_f', 'k_G', 'k_spring', 'kappa', 'kb', 'L', 'L_rad', 'lambd', 'm', 'M', 'm_0', 'm1', 'm2', 'mob', 'mom', 'mu', 'mu_drift', 'Nn', 'n', 'n_rho', 'n_0', 'omega', 'omega_0', 'p', 'Pol', 'p_d', 'Pwr', 'pr', 'prob', 'q', 'q1', 'q2', 'r', 'R', 'r1', 'r2', 'rho', 'rho_c', 'rho_c_0', 'rho_0', 'sigma', 'sigma_den', 't', 'T', 't1', 'T1', 'T2', 'tau', 'theta', 'theta1', 'theta2', 'U', 'u', 'v', 'V', 'v1', 'V1', 'V2', 'Volt', 'w', 'x', 'x1', 'x2', 'x3', 'y', 'Y', 'y1', 'y2', 'y3', 'z', 'z1', 'z2', 'mu_S', 'L_ind', 'Z_1', 'Z_2', 'Length', 'Mass', 'Temperature', 'Voltage', 'Τime', (-3.0, 0.0, 0.0, 0.0, 0.0), (-3.0, 0.0, 1.0, 0.0, 0.0), (-2.0, 1.0, 0.0, 0.0, 1.0), (-2.0, 3.0, -1.0, 0.0, 2.0), (-2.0, 4.0, -1.0, 0.0, 2.0

Creating the list of formulas, which we use for testing

In [3]:
all_mysteries = KR.import_equations()

First perform a small scale experiment. we consider all dimensionless variables

In [4]:
print([(x,y,z) for x,y,z in G.edges(data = True) if z.get("IS_VARIABLE_IN") == True and x in
       [(x) for x,y,z in G.edges(data = True) if y== (0,0,0,0,0) and z.get("HAS_UNIT") == True]])

[('alpha', '((x1*(cos(omega*t)+alpha*cos(omega*t)**2)))', {'IS_VARIABLE_IN': True}), ('alpha', '((n*alpha/(1-(n*alpha/3))*epsilon*Ef))', {'IS_VARIABLE_IN': True}), ('alpha', '((1+n*alpha/(1-(n*alpha/3))))', {'IS_VARIABLE_IN': True}), ('alpha', '((mom*H/(kb*T)+(mom*alpha)/(epsilon*c**2*kb*T)*M))', {'IS_VARIABLE_IN': True}), ('alpha', '((2*pi*alpha/(n*d)))', {'IS_VARIABLE_IN': True}), ('alpha', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('beta', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('chi', '((sigma_den/epsilon*1/(1+chi)))', {'IS_VARIABLE_IN': True}), ('chi', '((mom*(1+chi)*B))', {'IS_VARIABLE_IN': True}), ('delta', '((I1+I2+2*sqrt(I1*I2)*cos(delta)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*q*B/(2*m)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*mom*B*Jz/(h/(2*pi))))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*pr*V))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*kb*v/A))', {'IS_VARIABLE_IN': True}), ('gamma', '((sqrt(gamma*pr/rho))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass


In [6]:
def generate_differentials_at_point(point, eqn):
    x = tf.Variable(point, dtype = float)  # Create a Tensorflow variable initialized to 1.0

    with tf.GradientTape() as t3: 
        with tf.GradientTape() as t2:
            with tf.GradientTape() as t1:
                y = eval(eqn)

      # Compute the gradient inside the outer `t2` context manager
      # which means the gradient computation is differentiable as well.
            dy_dx = t1.gradient(y, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
        d2y_dx2 = t2.gradient(dy_dx, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    d3y_dx3 = t3.gradient(d2y_dx2, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    return [y.numpy(), dy_dx.numpy(), d2y_dx2.numpy(), d3y_dx3.numpy()]

def generate_taylor_series(arr):
    series = []
    for i in range(len(arr)):
        series.append(arr[i]/math.factorial(i))
    return series

# print((generate_differentials_at_point(1, tf.math.sin)))
# print(generate_taylor_series(generate_differentials_at_point(1, tf.math.sin)))

In [7]:
import automatic_differentiation as AD

def local_power(y):
    def pow(x):
        return tf.math.pow(x,y)
    return pow

point = 1

equation_to_alter = "(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))"

list_of_equations = ["tf.math.sin(x)", "tf.math.cos(x)", "tf.math.log(x)", "tf.math.exp(x)", "(x**2)",
                     "tf.math.sqrt(x)", "x**3", "tf.math.sinh(x)", "tf.math.cosh(x)", "tf.math.reciprocal(x)"]

list_compositionality = ["tf.math.sin(", "tf.math.cos(", "tf.math.log(", "tf.math.exp(", 
                     "tf.math.sqrt(", "tf.math.sinh(", "tf.math.cosh(", "tf.math.reciprocal("]

all_possible_equations = []

for i in list_compositionality:
    all_possible_equations.append(i+equation_to_alter+")")
        

all_possible_equations.append(equation_to_alter + "**3")
all_possible_equations.append(equation_to_alter + "**2")


for j in list_of_equations:
    all_possible_equations.append(equation_to_alter+"*"+j)
    all_possible_equations.append(equation_to_alter+"+"+j)
    all_possible_equations.append(equation_to_alter+"-"+j)
    all_possible_equations.append(equation_to_alter+"/"+j)

print(all_possible_equations)

taylor_series_ground_truth = []

for i in all_possible_equations:
    print(i)
    taylor_series_ground_truth.append(generate_taylor_series(generate_differentials_at_point(point, i)))
    
print(taylor_series_ground_truth)

def maape(actual, pred):
    return np.mean(np.arctan(np.abs((np.array(actual) - np.array(pred)) / np.array(pred)))) * 100

from sklearn.metrics import mean_squared_error
    

['tf.math.sin((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.cos((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.log((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.exp((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.sqrt((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.sinh((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.cosh((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', 'tf.math.reciprocal((tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi)))', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))**3', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))**2', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))*tf.math.sin(x)', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))+tf.math.sin(x)', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))-tf.math.sin(x)', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))/tf.math.sin(x)', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))*tf.math.cos(x)', '(tf.math.exp(-x**2/2)/tf.math.sqrt(2*math.pi))+tf.mat

In [8]:
# Test equations have the following taylor series:
# print(taylor_series_ground_truth[6])    
# print(taylor_series_ground_truth[16])    
# print(taylor_series_ground_truth[26])    
# print(taylor_series_ground_truth[36])    
# print(taylor_series_ground_truth[45])    
# print(taylor_series_ground_truth[56])    
# print(taylor_series_ground_truth[66])    
# print(taylor_series_ground_truth[76])    
# print(taylor_series_ground_truth[86])    
# print(taylor_series_ground_truth[96])    

equation_to_alter = "(np.exp(-x**2/2)/np.sqrt(2*math.pi))"

list_of_equations_np = ["np.sin(x)", "np.cos(x)", "np.log(x)", "np.exp(x)", "(x**2)",
                     "np.sqrt(x)", "x**3", "np.sinh(x)", "np.cosh(x)", "np.reciprocal(x)"]   

list_compositionality_np = ["np.sin(", "np.cos(", "np.log(", "np.exp(", 
                     "np.sqrt(", "np.sinh(", "np.cosh(", "np.reciprocal("]

all_possible_equations_np = []

for i in list_compositionality_np:
    all_possible_equations_np.append(i+equation_to_alter+")")
        

all_possible_equations_np.append(equation_to_alter + "**3")
all_possible_equations_np.append(equation_to_alter + "**2")


for j in list_of_equations_np:
    all_possible_equations_np.append(equation_to_alter+"*"+j)
    all_possible_equations_np.append(equation_to_alter+"+"+j)
    all_possible_equations_np.append(equation_to_alter+"-"+j)
    all_possible_equations_np.append(equation_to_alter+"/"+j)



# for i in list_compositionality:
#     for j in list_of_equations:
#         all_possible_equations_np.append(i+j+")")
        
# for i in list_of_equations:
#     all_possible_equations_np.append("(" + i + ")**3")
#     all_possible_equations_np.append("(" + i + ")**2")

    
# for i in list_of_equations_np:
#     for j in list_of_equations_np:
#         all_possible_equations_np.append(i+"*"+j)
#         all_possible_equations_np.append(i+"+"+j)
#         all_possible_equations_np.append(i+"-"+j)
#         all_possible_equations_np.append(i+"/"+j)

print(len(all_possible_equations_np))

50


In [9]:
total_results_mse = []
total_results_maape = []

for test in range(50):
    
    try:
        x = np.random.rand(10000)*2+1
        y = eval(all_possible_equations_np[test])


        results = []

        for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
            results.append([np.array(AD.multiple_gradients(x, y, point = point, activation = i, learning_rate = 0.01)).reshape(4)])
        results.append(np.array(AD.three_grads(x, y, point = point)).reshape(4))   
        
        activation_rank_mse = []
        activation_rank_maape = []
        for i in results:
            print("predicting using function")
            mse_results = []
            maape_results = []
            for j in taylor_series_ground_truth:
                try:
                    mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
                except:
                    mse_results.append(100000000)
                try:
                    maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
                except:
                    maape_results.append(100000000)
            rank_mse = np.array(mse_results).argsort().argsort()[test]+1
            rank_maape = np.array(maape_results).argsort().argsort()[test]+1
            print("suggests the following with mse: ", rank_mse)
            print("suggests the following with maape: ", rank_maape)
            activation_rank_mse.append(rank_mse)
            activation_rank_maape.append(rank_maape)
        total_results_mse.append(activation_rank_mse)
        total_results_maape.append(activation_rank_maape)
        
    except:
        total_results_mse.append([100000000,100000000,100000000,100000000,100000000])
        total_results_maape.append([100000000,100000000,100000000,100000000,100000000])   

Epoch 1/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 3/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0275
Epoch 4/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0236
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0205
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0184
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0162
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0150
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0136
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0128
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0117
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0111
Epoch 13/20
79/79 [==============================

79/79 [==============================] - 0s 1ms/step - loss: 0.0107
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0083
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0069
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0047
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0041
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0038
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 13/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 14/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 15/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0027
Epoch 16/20
79/79 [==============================] - 0s 1

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/v

79/79 [==============================] - 0s 1ms/step - loss: 0.0856
Epoch 2/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0252
Epoch 3/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0210
Epoch 4/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0173
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0147
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0122
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0106
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0093
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0082
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0070
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0062
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0056
Epoch 13/20
79/79 [==============================] - 0s 1ms/

79/79 [==============================] - 0s 1ms/step - loss: 0.0094
Epoch 4/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0073
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0062
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0048
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0041
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0028
Epoch 13/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0027
Epoch 14/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 15/20
79/79 [==============================] - 0s 1m

...

79/79 [==============================] - 0s 1ms/step - loss: 0.0095
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0079
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0067
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0060
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0048
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0045
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0040
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0039
Epoch 13/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 14/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 15/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 16/20
79/79 [==============================] - 0s 1

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/v

79/79 [==============================] - 0s 1ms/step - loss: 0.0464
Epoch 2/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0360
Epoch 3/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0302
Epoch 4/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0262
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0233
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0205
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0195
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0171
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0158
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0143
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0139
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0128
Epoch 13/20
79/79 [==============================] - 0s 1ms/

79/79 [==============================] - 0s 1ms/step - loss: 0.0116
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0100
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0084
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0074
Epoch 8/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0070
Epoch 9/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0060
Epoch 10/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0055
Epoch 11/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0050
Epoch 12/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0046
Epoch 13/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0044
Epoch 14/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0042
Epoch 15/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0037
Epoch 16/20
79/79 [==============================] - 0s 1

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.pyenv/v

In [10]:
print(total_results_mse)
print(total_results_maape)

[[5, 5, 4, 5, 6], [1, 1, 1, 1, 1], [1, 1, 4, 1, 1], [4, 4, 3, 4, 4], [12, 12, 2, 12, 2], [7, 7, 5, 7, 8], [2, 2, 2, 1, 1], [18, 33, 1, 1, 33], [1, 1, 1, 1, 1], [2, 2, 2, 2, 2], [8, 7, 9, 8, 10], [8, 14, 7, 8, 5], [13, 9, 7, 9, 5], [20, 20, 19, 20, 19], [12, 12, 11, 12, 12], [31, 30, 1, 32, 29], [27, 23, 1, 27, 17], [31, 31, 32, 32, 32], [15, 14, 15, 15, 15], [27, 27, 14, 27, 25], [27, 29, 17, 28, 29], [50, 50, 50, 50, 50], [13, 16, 5, 14, 3], [1, 1, 1, 1, 8], [1, 3, 1, 1, 8], [3, 3, 3, 3, 3], [12, 12, 15, 12, 13], [9, 2, 1, 10, 33], [9, 3, 1, 9, 35], [40, 40, 39, 40, 40], [4, 4, 6, 4, 4], [2, 1, 1, 2, 3], [1, 1, 1, 1, 1], [14, 14, 13, 14, 14], [18, 18, 20, 18, 19], [28, 30, 39, 17, 22], [32, 7, 46, 28, 28], [47, 47, 47, 47, 47], [8, 9, 12, 8, 5], [3, 1, 3, 5, 9], [5, 1, 2, 6, 22], [22, 22, 21, 22, 22], [1, 1, 1, 3, 1], [1, 1, 4, 1, 4], [2, 1, 4, 2, 4], [9, 9, 7, 9, 10], [21, 21, 20, 21, 21], [36, 36, 32, 36, 35], [32, 33, 28, 32, 28], [7, 6, 10, 6, 6]]
[[7, 7, 16, 8, 4], [1, 1, 1, 1, 2

In [11]:
np.savetxt('embeddings_twostep_onedim_mse_graphEG.csv', total_results_mse, delimiter=',', fmt = '%s') 
np.savetxt('embeddings_twostep_onedim_maape_graphEG.csv', total_results_maape, delimiter=',', fmt = '%s') 